## Deliverable 2. Create a Customer Travel Destinations Map.

In [14]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [15]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Metro,ID,2021-01-18 01:17:15,-5.1131,105.3067,78.13,84,73,11.97,broken clouds
1,1,Rikitea,PF,2021-01-18 01:17:15,-23.1203,-134.9692,78.87,75,31,12.17,scattered clouds
2,2,Lasa,IT,2021-01-18 01:17:15,46.6166,10.7002,28.00,80,75,9.22,broken clouds
3,3,Bonavista,CA,2021-01-18 01:17:15,48.6499,-53.1147,39.00,93,100,4.61,moderate rain
4,4,Souillac,MU,2021-01-18 01:05:38,-20.5167,57.5167,78.80,83,40,11.50,scattered clouds


In [16]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 100


In [17]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & 
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Metro,ID,2021-01-18 01:17:15,-5.1131,105.3067,78.13,84,73,11.97,broken clouds
1,1,Rikitea,PF,2021-01-18 01:17:15,-23.1203,-134.9692,78.87,75,31,12.17,scattered clouds
4,4,Souillac,MU,2021-01-18 01:05:38,-20.5167,57.5167,78.80,83,40,11.50,scattered clouds
5,5,Faanui,PF,2021-01-18 01:11:51,-16.4833,-151.7500,82.08,74,0,12.39,clear sky
8,8,Soe,ID,2021-01-18 01:17:16,-9.8607,124.2840,76.69,77,32,3.58,light rain
10,10,Vaini,TO,2021-01-18 01:17:17,-21.2000,-175.2000,87.80,74,75,11.50,broken clouds
17,17,Cabo San Lucas,MX,2021-01-18 01:08:03,22.8909,-109.9124,73.40,56,1,5.53,clear sky
18,18,Srikakulam,IN,2021-01-18 01:17:18,18.3000,83.9000,69.12,82,35,5.84,scattered clouds
19,19,Mar Del Plata,AR,2021-01-18 01:17:18,-38.0023,-57.5575,62.60,77,75,9.22,broken clouds
22,22,Hermanus,ZA,2021-01-18 01:11:33,-34.4187,19.2345,66.99,80,0,7.00,clear sky


In [18]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID        0
City           0
Country        3
Date           0
Lat            0
Lng            0
Max Temp       0
Humidity       0
Cloudiness     0
Wind Speed     0
Description    0
dtype: int64

In [20]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df[preferred_cities_df['Country'].notna()]

In [10]:
clean_df
clean_df.isnull().sum()

City_ID        0
City           0
Country        0
Date           0
Lat            0
Lng            0
Max Temp       0
Humidity       0
Cloudiness     0
Wind Speed     0
Description    0
dtype: int64

In [22]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,Metro,ID,78.13,broken clouds,-5.1131,105.3067,
1,Rikitea,PF,78.87,scattered clouds,-23.1203,-134.9692,
4,Souillac,MU,78.80,scattered clouds,-20.5167,57.5167,
5,Faanui,PF,82.08,clear sky,-16.4833,-151.7500,
8,Soe,ID,76.69,light rain,-9.8607,124.2840,
10,Vaini,TO,87.80,broken clouds,-21.2000,-175.2000,
17,Cabo San Lucas,MX,73.40,clear sky,22.8909,-109.9124,
18,Srikakulam,IN,69.12,scattered clouds,18.3000,83.9000,
19,Mar Del Plata,AR,62.60,broken clouds,-38.0023,-57.5575,
22,Hermanus,ZA,66.99,clear sky,-34.4187,19.2345,


In [23]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    try:
    # 6c. Get latitude and longitude from DataFrame
        lat = row["Lat"]
        lng = row["Lng"]
        params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
        hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except:  
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [29]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.isnull().sum()

City           0
Country        0
Max Temp       0
Description    0
Lat            0
Lng            0
Hotel Name     0
dtype: int64

In [30]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [36]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Max Temp} °F and {Description}</dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [37]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))